- Q!: 如何使用用户标签来指导业务（如何提升业务）: 标签化的第一目的就是通过用户画像与我们的业务相关联，提升业务的转化率， 减少目标客户的流失。 通常对标签的管理涉及到了主要三个层面对标签何其产生数据的管理和配置： 数据层，算法层，业务层。 在三个层面上分别都具有使用用户标签的方式提升业务的各个方面。a. 数据层是对标签数据的事实状态，因此更加清晰更加明确地存储架构会让后续的算法建模，业务逻辑处理具有十分重要的作用（ 用户行为的记录是否清晰， 用户消费的具体记录，清晰明细的用户标签对用户归类产生很大影响。 b. 随之明确地用户标签记录在算法层对用户数据的建模就更好地预测，归类， 推荐用户更加适合用户的内容商品偏好。 c. 通过建模，使用对预测 归类 的建模结果从而推出更加合适合理吸引用户的消费方式， 商品推荐类型，从而使用户使用本项目的时间更长，消费更多，从而提升项目业务的变现转化率，客户留存率等等。

- Q2: 对于数据打标签的方法 ：基于用户打标签。 基于内容打标签。   实际算法可用 隔各个聚类方法: kmeans  em gmm 等等

- Q3 : 准确率和精确率有何不同 : 
    P精确率是针对我们预测结果的预测准确率 是预测正确的数量和实际正确数量的比
    A准确率是测试数据集，分类器正确分类的样本数与总样本数之比。
    
- Q4: 用户标签的原理: (根据课程) 标签设计像可以用用户兴趣、偏好、人口统计学特征，主要目的是提升营销精准度、推荐匹配度，终极目的是提升产品服务，起到提升企业利润。用户画像适合于各个产品周期：从新用户的引流到潜在用户的挖掘、从老用户的培养到流失用户的回流等目标去查找利用各方面的标签，比如在q1 中说的用户行为等等。    餐厅的话，比如就餐环境，菜品， 服务态度，餐馆位置等等。


- Q5: 这10这种方式都是单独单例的方法。下一步可以尝试浅层融合： 简单粗暴，将最后的prediction 结果 数值化后 取均值。   深层次 可以尝试将多个弱的分类器转化成强分类器 （ adaboost 泛化集成） 

In [8]:
#titanic 
from tpot import TPOTClassifier 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
import xgboost as xgb

In [2]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


- Notice: age   Fare  embarked should be revised     cabin is so little

In [3]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
train_data.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Cameron, Miss. Clear Annie",male,1601,B96 B98,S
freq,1,577,7,4,644


In [9]:
def data():
    
    # Importing the dataset
    X_train = pd.read_csv('./train.csv')
    X_test = pd.read_csv('./test.csv')
    
    # Replace names with titles
    X_train['Name'] = X_train['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())
    titles = X_train['Name'].unique()
    
    X_test['Name'] = X_test['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())
    titles = X_test['Name'].unique()
    
    # Replace missing ages with title median
    X_train['Age'].fillna(-1, inplace=True)
    X_test['Age'].fillna(-1, inplace=True)
    
    medians = dict()
    for title in titles:
        median = X_train.Age[(X_train["Age"] != -1) & (X_train['Name'] == title)].median()
        medians[title] = median
        
    for index, row in X_train.iterrows():
        if row['Age'] == -1:
            X_train.loc[index, 'Age'] = medians[row['Name']]
    
    for index, row in X_test.iterrows():
        if row['Age'] == -1:
            X_test.loc[index, 'Age'] = medians[row['Name']]
            
    # Replace titles with numerical values
    replacement = {
        'Don': 0,
        'Rev': 0,
        'Jonkheer': 0,
        'Capt': 0,
        'Mr': 1,
        'Dr': 2,
        'Col': 3,
        'Major': 3,
        'Master': 4,
        'Miss': 5,
        'Mrs': 6,
        'Mme': 7,
        'Ms': 7,
        'Mlle': 7,
        'Sir': 7,
        'Lady': 7,
        'the Countess': 7
    }
    X_train['Name'] = X_train['Name'].apply(lambda x: replacement.get(x))        
    X_test['Name'] = X_test['Name'].apply(lambda x: replacement.get(x))  
     
    # Replace missing fare with class median
    X_train['Fare'].fillna(-1, inplace=True)
    X_test['Fare'].fillna(-1, inplace=True)
    
    medians = dict()
    for pclass in X_train['Pclass'].unique():
        median = X_train.Fare[(X_train["Fare"] != -1) & (X_train['Pclass'] == pclass)].median()
        medians[pclass] = median
        
    for index, row in X_train.iterrows():
        if row['Fare'] == -1:
            X_train.loc[index, 'Fare'] = medians[row['Pclass']]
    
    for index, row in X_test.iterrows():
        if row['Fare'] == -1:
            X_test.loc[index, 'Fare'] = medians[row['Pclass']]
    
    replacement = {
        6: 0,
        4: 0,
        5: 1,
        0: 2,
        2: 3,
        1: 4,
        3: 5
    }
    X_train['Parch'] = X_train['Parch'].apply(lambda x: replacement.get(x))
    X_test['Parch'] = X_test['Parch'].apply(lambda x: replacement.get(x))
    
    X_train['Embarked'] = X_train['Embarked'].fillna('S')
    X_test['Embarked'] = X_test['Embarked'].fillna('S')
    
    replacement = {
        'S': 0,
        'Q': 1,
        'C': 2
    }
    
    X_train['Embarked'] = X_train['Embarked'].apply(lambda x: replacement.get(x))
    X_test['Embarked'] = X_test['Embarked'].apply(lambda x: replacement.get(x))
    replacement  = {
        5: 0,
        8: 0,
        4: 1,
        3: 2,
        0: 3,
        2: 4,
        1: 5
    }
    
    X_train['SibSp'] = X_train['SibSp'].apply(lambda x: replacement.get(x))
    X_test['SibSp'] = X_test['SibSp'].apply(lambda x: replacement.get(x))
    
    X_train['Cabin'] = X_train['Cabin'].fillna('U')
    X_test['Cabin'] = X_test['Cabin'].fillna('U')
    
    # Retain first letter only of cabin
    X_train['Cabin'] = X_train['Cabin'].map(lambda x: x[0])
    X_test['Cabin'] = X_test['Cabin'].map(lambda x: x[0])
    
    replacement = {
        'T': 0,
        'U': 1,
        'A': 2,
        'G': 3,
        'C': 4,
        'F': 5,
        'B': 6,
        'E': 7,
        'D': 8
    }
    
    X_train['Cabin'] = X_train['Cabin'].apply(lambda x: replacement.get(x))
    X_test['Cabin'] = X_test['Cabin'].apply(lambda x: replacement.get(x))
    
    X_train['Sex'] = LabelEncoder().fit_transform(X_train['Sex'])
    X_test['Sex'] = LabelEncoder().fit_transform(X_test['Sex'])
    
    y_train = X_train.iloc[:, 1].values
    submission = X_test.iloc[:, 0].values
    submission = pd.DataFrame(submission) 
    submission.columns = ['PassengerId'] 
    
    # Delete redundant features
    X_train = X_train.drop(X_train.columns[[1, 8]], axis=1)
    X_test = X_test.drop(X_test.columns[[7]], axis=1)

    return X_train, y_train, X_test, submission

X_train, y_train, X_test, submission = data()
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,3,1,1,22.0,5,2,7.2500,1,0
1,2,1,6,0,38.0,5,2,71.2833,4,2
2,3,3,5,0,26.0,3,2,7.9250,1,0
3,4,1,6,0,35.0,5,2,53.1000,4,0
4,5,3,1,1,35.0,3,2,8.0500,1,0
...,...,...,...,...,...,...,...,...,...,...
886,887,2,0,1,27.0,3,2,13.0000,1,0
887,888,1,5,0,19.0,3,2,30.0000,6,0
888,889,3,5,0,21.0,5,3,23.4500,1,0
889,890,1,1,1,26.0,3,2,30.0000,4,2


In [11]:
X_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,892,3,1.0,1,34.5,3,2.0,7.8292,1,1
1,893,3,6.0,0,47.0,5,2.0,7.0000,1,0
2,894,2,1.0,1,62.0,3,2.0,9.6875,1,1
3,895,3,1.0,1,27.0,3,2.0,8.6625,1,0
4,896,3,6.0,0,22.0,5,4.0,12.2875,1,0
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1.0,1,30.0,3,2.0,8.0500,1,0
414,1306,1,NaN,0,39.0,3,2.0,108.9000,4,2
415,1307,3,1.0,1,38.5,3,2.0,7.2500,1,0
416,1308,3,1.0,1,30.0,3,2.0,8.0500,1,0


In [10]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=2)
tpot.fit(X_train, y_train)

y_pred = tpot.predict(X_test)
y_pred = pd.DataFrame(y_pred) 
y_pred.columns = ['Survived'] 
submission = submission.join(y_pred) 
submission
#too slow limit to 2mins XD!

Version 0.10.2 of tpot is outdated. Version 0.11.6 was released Monday October 26, 2020.


Generation 1 - Current best internal CV score: 0.8384580466538785

2.04532315 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(Nystroem(ExtraTreesClassifier(FastICA(input_matrix, tol=0.8500000000000001), bootstrap=True, criterion=entropy, max_features=0.35000000000000003, min_samples_leaf=4, min_samples_split=15, n_estimators=100), gamma=0.55, kernel=poly, n_components=5), bootstrap=False, criterion=gini, max_features=0.4, min_samples_leaf=18, min_samples_split=12, n_estimators=100)
Imputing missing values in feature set


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [12]:
#so RandomForest
import sklearn.ensemble as ske
# Todo: randomforest 1,0 ?
results_rf = ske.RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
results_rf.score(X_train, y_train)

1.0